### **Reconhecedor de padrões baseado em LZW**

#### Pré-processamento das imagens

##### 1. Retirada do cabeçalho dos dados

In [66]:
import os

In [67]:
# Diretório de origem
diretorio_origem = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces"

# Diretório de destino
diretorio_destino = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_sc"

In [68]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino
    caminho_destino_subpasta = os.path.join(diretorio_destino, subpasta)
    os.makedirs(caminho_destino_subpasta, exist_ok=True)

    # Percorre os arquivos de imagem na subpasta atual
    for arquivo in os.listdir(caminho_subpasta):
        if arquivo.endswith(".pgm"):
            caminho_arquivo_origem = os.path.join(caminho_subpasta, arquivo)

            # Abre o arquivo de origem em modo binário
            with open(caminho_arquivo_origem, "rb") as arquivo_origem:
                dados = arquivo_origem.read()

            # Remove as três primeiras linhas
            novos_dados = dados.split(b"\n", 3)[-1]

            # Cria o caminho do arquivo de destino
            caminho_arquivo_destino = os.path.join(caminho_destino_subpasta, arquivo)

            # Salva os novos dados no arquivo de destino em modo binário
            with open(caminho_arquivo_destino, "wb") as arquivo_destino:
                arquivo_destino.write(novos_dados)

#### Geração dos modelos

##### 1. Divisão aleatória do banco entre instâncias de "treinamento" e "teste" - **ocorre a cada novo valor de k**

In [184]:
import shutil
import random

In [185]:
# Diretório de origem
diretorio_origem = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_sc"

# Diretório de destino para treinamento
diretorio_destino_treinamento = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_div/train"

# Diretório de destino para teste
diretorio_destino_teste = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_div/test"

# Porcentagem de imagens para treinamento (90%)
porcentagem_treinamento = 0.9

In [186]:
# Exclui os diretórios de destino, se já existirem
if os.path.exists(diretorio_destino_treinamento):
    shutil.rmtree(diretorio_destino_treinamento)
if os.path.exists(diretorio_destino_teste):
    shutil.rmtree(diretorio_destino_teste)

In [187]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta_origem = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino para treinamento
    caminho_subpasta_destino_treinamento = os.path.join(diretorio_destino_treinamento, subpasta)
    os.makedirs(caminho_subpasta_destino_treinamento)

    # Cria a subpasta correspondente no diretório de destino para teste
    caminho_subpasta_destino_teste = os.path.join(diretorio_destino_teste, subpasta)
    os.makedirs(caminho_subpasta_destino_teste)

    # Lista os arquivos na subpasta atual
    arquivos = os.listdir(caminho_subpasta_origem)
    random.shuffle(arquivos)  # Embaralha a ordem dos arquivos

    # Calcula o número de arquivos para treinamento e teste
    num_arquivos_treinamento = int(len(arquivos) * porcentagem_treinamento)
    num_arquivos_teste = len(arquivos) - num_arquivos_treinamento

    # Divide os arquivos entre treinamento e teste
    arquivos_treinamento = arquivos[:num_arquivos_treinamento]
    arquivos_teste = arquivos[num_arquivos_treinamento:]

    # Move os arquivos para as respectivas pastas de treinamento e teste
    for arquivo in arquivos_treinamento:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_treinamento, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

    for arquivo in arquivos_teste:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_teste, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

##### 2. Construção dos modelos/dicionários para cada classe

In [188]:
dict_size = 256

In [189]:
def lzw_encode(input_file, dictionary, max_dict_size):
    current_sequence = bytes()
    global dict_size

    with open(input_file, "rb") as file:
        while True:
            symbol = file.read(1)
            if not symbol:
                break

            next_sequence = current_sequence + symbol
            if next_sequence in dictionary:
                current_sequence = next_sequence
            else:
                if dict_size < 2 ** max_dict_size:
                    dictionary[next_sequence] = dict_size
                    dict_size += 1
                current_sequence = symbol

In [190]:
def encode_images_in_folders(folder_path, max_dict_size):
    dictionaries = []

    # Verifica se o tamanho máximo do dicionário está dentro do intervalo válido
    if not (9 <= max_dict_size <= 16):
        raise ValueError("O tamanho máximo do dicionário deve estar entre 9 e 16.")

    # Percorre todas as pastas
    for folder_name in os.listdir(folder_path):
        folder_dir = os.path.join(folder_path, folder_name)
        dictionary = {bytes([i]): i for i in range(256)}

        # Codifica as imagens na pasta atual
        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)

            # Realiza a codificação LZW para cada imagem
            lzw_encode(image_path, dictionary, max_dict_size)
        
        global dict_size
        dict_size = 256

        # Adiciona o dicionário da pasta à lista de dicionários
        dictionaries.append((folder_name, dictionary))

    return dictionaries

In [191]:
# Diretório das pastas de imagens
folder_path = diretorio_destino_treinamento
# Tamanho máximo do dicionário (2^k)
max_dict_size = 15
# Codifica as imagens em todas as pastas e obtém a lista de dicionários
all_dictionaries = encode_images_in_folders(folder_path, max_dict_size)

In [210]:
folder_name, dictionary = all_dictionaries[3]
print(f"Dicionário da pasta '{folder_name}'")
print(f"Tamanho: {len(dictionary)}")

Dicionário da pasta 's30':
Tamanho: 32768


##### 3. Classificação das imagens de teste

In [193]:
def lzw_encode(input_file, dictionary):
    current_sequence = bytes()
    encoded_data = []

    with open(input_file, "rb") as file:
        for symbol in file.read():
            next_sequence = current_sequence + bytearray([symbol])
            if bytes(next_sequence) in dictionary[1]:
                current_sequence = next_sequence
            else:
                encoded_data.append(dictionary[1][bytes(current_sequence)])
                current_sequence = bytearray([symbol])

    encoded_data.append(dictionary[1][bytes(current_sequence)])
    return encoded_data

In [194]:
import struct

In [62]:
def encode_images_in_folders(folder_path, dictionaries, output_folder):

    os.makedirs(output_folder, exist_ok=True)

    for folder_name in os.listdir(folder_path):
        folder_dir = os.path.join(folder_path, folder_name)
        encoded_data = []

        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)

            for dictionary in dictionaries:
                encoded_data.extend(lzw_encode(image_path, dictionary))
                encoded_file_path = os.path.join(output_folder, f"{folder_name}_{image_name}_dict_{dictionary[0]}.bin")
                
                with open(encoded_file_path, "wb") as output_file:
                    for value in encoded_data:
                        output_file.write(struct.pack("H", value))

In [196]:
# Definir os caminhos das pastas de entrada, onde estão localizadas as imagens
folder_path = diretorio_destino_teste

# Caminho para salvar os arquivos binários gerados para cada valor de k
output_folder = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/output/k_15"

# Lista de tuplas contendo os dicionários e suas classes associadas
dictionaries = all_dictionaries

In [197]:
# Codificar as imagens nas pastas usando os dicionários especificados
encode_images_in_folders(folder_path, dictionaries, output_folder)

##### 4. Avaliação do modelo para cada valor de k

In [208]:
def calculate_counts(output_folder):
    counts = [0] * 8  # Lista para armazenar os contadores das subpastas k_i (índices de 1 a 8)

    for i in range(1, 41):
        prefix = f"s{i}_"

        for j in range(1, 41):
            dict_file_name = f"{prefix}{j}.pgm_dict_s{i}.bin"
            dict_file_path = os.path.join(output_folder, dict_file_name)

            # Verifica se o arquivo dict_i existe
            if os.path.exists(dict_file_path):
                dict_size = os.path.getsize(dict_file_path)

                # Verifica se o arquivo dict_i é o menor entre os demais arquivos do mesmo prefixo
                for k in range(1, 41):
                    file_name = f"{prefix}{k}.pgm_dict_s{j}.bin"
                    file_path = os.path.join(output_folder, file_name)

                    if os.path.exists(file_path) and os.path.getsize(file_path) < dict_size:
                        break
                else:
                    counts[j - 9] += 1

    # Divide os contadores por 40
    counts = [round(count / 40, 4) for count in counts]
    return counts

In [209]:
# Exemplo de uso
output_folder = output_folder
counts = calculate_counts(output_folder)

for i, count in enumerate(counts):
    subfolder_name = f"k_{i + 9}"
    print(f"A subpasta {subfolder_name} possui {count * 40} arquivo(s) codificado(s) 'corretamente', com acurácia {count}")

A subpasta k_9 possui 2.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.05
A subpasta k_10 possui 3.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.075
A subpasta k_11 possui 4.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.1
A subpasta k_12 possui 3.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.075
A subpasta k_13 possui 1.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.025
A subpasta k_14 possui 2.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.05
A subpasta k_15 possui 0.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.0
A subpasta k_16 possui 0.0 arquivo(s) codificado(s) 'corretamente', com acurácia 0.0
